In [0]:
-- Query to analyze ML model serving consumption metrics across workspaces
WITH model_serving_usage AS (
    SELECT 
        usage_date,
        usage_metadata.endpoint_name AS endpoint_name,
        sku_name,
        billing_origin_product,
        -- Extract relevant custom tags for better analysis
        custom_tags,
        usage_quantity,
        usage_unit,
        usage_type,
        usage_start_time,
        usage_end_time,
        workspace_id,
        cloud,
        account_id,
        usage_metadata,
        product_features
    FROM system.billing.usage u
    WHERE 
        -- Filter for ML model serving related usage only
        contains(sku_name, 'SERVERLESS_REAL_TIME_INFERENCE')
        AND billing_origin_product = 'MODEL_SERVING'
        AND usage_date BETWEEN :p_start_date AND :p_end_date
        -- Dynamic filtering based on user parameters
        AND (:p_endpoint = '--all--' OR contains(usage_metadata.endpoint_name, :p_endpoint))
        AND (:workspace_id = '--all--' OR contains(workspace_id, :workspace_id))
)

SELECT 
    u.usage_date,
    u.endpoint_name,
    u.sku_name,
    u.billing_origin_product,
    u.custom_tags,
    u.usage_quantity,
    u.usage_unit,
    u.usage_type,
    -- Calculate costs
    lp.pricing.default AS list_price,
    lp.pricing.default * u.usage_quantity AS list_cost,
    -- Additional time-based dimensions
    date_format(u.usage_date, 'yyyy-MM') AS year_month,
    u.usage_start_time,
    u.usage_end_time,
    u.workspace_id,
    u.cloud,
    u.account_id,
    u.usage_metadata,
    u.product_features
FROM model_serving_usage u
INNER JOIN system.billing.list_prices lp 
    ON  u.cloud = lp.cloud 
    AND u.sku_name = lp.sku_name 
    AND u.usage_start_time >= lp.price_start_time
ORDER BY u.usage_start_time DESC;